### Using Scispacy

In [1]:
### chose to do this from cli
#%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

In [2]:
import pandas as pd

In [30]:
patient_notes = pd.read_csv("patient_notes.csv")
features = pd.read_csv("features.csv")

print(patient_notes[patient_notes["case_num"] == 0][:10])
features[features["case_num"] == 0][:10]

   pn_num  case_num                                         pn_history
0       0         0  17-year-old male, has come to the student heal...
1       1         0  17 yo male with recurrent palpitations for the...
2       2         0  Dillon Cleveland is a 17 y.o. male patient wit...
3       3         0  a 17 yo m c/o palpitation started 3 mos ago; \...
4       4         0  17yo male with no pmh here for evaluation of p...
5       5         0  17 yo m, presenting with palpitations/increase...
6       6         0  17 yo male presents with 3-4 month history of ...
7       7         0  17 yo male presents with a 2-3 month history o...
8       8         0  HPI: 17 YO complains of intermittent heart pal...
9       9         0  Mr. Cleveland is a 17 yo M who presents to cli...


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded
5,5,0,No-hair-changes-OR-no-nail-changes-OR-no-tempe...
6,6,0,Adderall-use
7,7,0,Shortness-of-breath
8,8,0,Caffeine-use
9,9,0,heart-pounding-OR-heart-racing


In [4]:
case_num_zero = patient_notes["pn_history"][patient_notes["case_num"] == 0][:1500]
whole_case_num_zero = "\n".join(case_num_zero.to_list())
len(whole_case_num_zero)
whole_case_num_zero[:1000]

"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std\n17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating.

In [19]:
import prepare_jag
clean_note = prepare_jag.basic_clean3(whole_case_num_zero)
clean_note = prepare_jag.remove_stopwords(clean_note, exclude_words=["no"])
len(clean_note)

968544

### run this always

In [16]:
import scispacy
import spacy

from scispacy.linking import EntityLinker

sci_nlp = spacy.load("en_ner_bc5cdr_md")
# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
print("works, yay!")

works, yay!


### From Scispacy GitHub

In [ ]:
# # NOTE: The resolve_abbreviations parameter is optional, and requires that
# # the AbbreviationDetector pipe has already been added to the pipeline. Adding
# # the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# # that linking will only be performed on the long form of abbreviations.
# sci_nlp.add_pipe("scispacy_linker", config={"linker_name": "umls"})

# doc = sci_nlp(clean_note)
# # Let's look at a random entity!
# # doc comes from above
# entity = doc.ents[1]
# print(f"Name: {entity}")

# # Each entity is linked to UMLS with a score
# # (currently just char-3gram matching).
# linker = nlp.get_pipe("scispacy_linker")
# for umls_ent in entity._.kb_ents:
# 	print(linker.kb.cui_to_entity[umls_ent[0]])

### From https://gbnegrini.com/post/biomedical-text-nlp-scispacy-named-entity-recognition-medical-records/

In [ ]:
doc = sci_nlp(clean_note)
print("TEXT", "START", "END", "ENTITY TYPE")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

- case num 0 patient notes

In [26]:
case_num_zero_notes = patient_notes[patient_notes["case_num"] == 0]
case_num_zero_notes.head(3)

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...


- case num 0 feature text

In [31]:
case_num_zero_features = features[features["case_num"] == 0]
case_num_zero_features.head(3)

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure


### merge of the notes and features

In [32]:
notes_features = patient_notes.merge(right=features, left_on="case_num", right_on="case_num")#.drop("case_num_y", axis=1).rename({"case_num_x": "case_num"}, axis=1)
print(notes_features.shape)
notes_features.head()

(626902, 5)


,pn_num,case_num,pn_history,feature_num,feature_text
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded


### Create function to repeat this for each note in the dataframe

In [33]:
def get_doc_ents(clean_text):
    
    doc = sci_nlp(clean_text)
    # print("TEXT", "START", "END", "ENTITY TYPE")
    return {ent.text: [ent.start_char, ent.end_char] for ent in doc.ents}

In [34]:
notes_features["clean_pn_history"] = notes_features["pn_history"].apply(prepare_jag.basic_clean3).apply(prepare_jag.remove_stopwords, exclude_words=["no"])

notes_features["clean_notes_history_entities"] = notes_features["clean_pn_history"].apply(get_doc_ents)

In [35]:
notes_features.head()

,pn_num,case_num,pn_history,feature_num,feature_text,clean_pn_history,clean_notes_history_entities
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...,17-year-old male come student health clinic co...,"{'heart pounding': [56, 70], 'dispnea': [257, ..."
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder,17-year-old male come student health clinic co...,"{'heart pounding': [56, 70], 'dispnea': [257, ..."
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure,17-year-old male come student health clinic co...,"{'heart pounding': [56, 70], 'dispnea': [257, ..."
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms,17-year-old male come student health clinic co...,"{'heart pounding': [56, 70], 'dispnea': [257, ..."
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded,17-year-old male come student health clinic co...,"{'heart pounding': [56, 70], 'dispnea': [257, ..."


In [ ]:
notes_features["clean_notes_history_entities"].shape

In [ ]:
notes_features["clean_notes_history_entities"].head(50)

In [ ]:
case_num_feature_text = {}
for i in range(10):
    case_num_feature_text[i] = features["feature_text"][features["case_num"] == i].to_list()
case_num_feature_text

In [53]:
features[["case_num", "feature_num", "feature_text"]]

,case_num,feature_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,0,1,Family-history-of-thyroid-disorder
2,0,2,Chest-pressure
3,0,3,Intermittent-symptoms
4,0,4,Lightheaded
...,...,...,...
138,9,912,Family-history-of-migraines
139,9,913,Female
140,9,914,Photophobia
141,9,915,No-known-illness-contacts


### creating dictionary of dictionaries
- first dictionary key is the case_num
- first dictionary value (second dictionary key) is the feature_num
- second dictionary value is the feature_text

In [ ]:
# looking at feature_num per case_num
for i in features["case_num"].unique():
    print(i, features["feature_num"][features["case_num"] == i].values)

In [ ]:
# testing how to get only strings
for a in features["feature_num"][features["case_num"] == 0]:
    print(features["feature_text"][features["feature_num"] == a].to_string()[5:].strip())

In [ ]:
d = {}
for i in features["case_num"].unique():
    d[i] = {a: features["feature_text"][features["feature_num"] == a].to_string()[5:].strip() for a in features["feature_num"][features["case_num"] == i]}
d

In [194]:
df = pd.DataFrame(data=[d.keys(), d.values()]).T
df.columns = ["case_num", "features"]
df

,case_num,features
0,0,{0: 'Family-history-of-MI-OR-Family-history-of...
1,1,"{100: 'No-vaginal-discharge', 101: 'Weight-los..."
2,2,"{200: 'Prior-normal-periods', 201: 'Last-Pap-s..."
3,3,{300: 'FHx-of-PUD-OR-Family-history-of-peptic-...
4,4,"{400: 'Lack-of-other-thyroid-symptoms', 401: '..."
5,5,"{500: 'Onset-5-years-ago', 501: 'Female', 502:..."
6,6,"{600: 'Subjective-fevers', 601: 'Male', 602: '..."
7,7,"{700: 'Female', 701: 'Weight-Gain', 702: 'heav..."
8,8,"{800: 'Increased-appetite', 801: 'Son-died-3-w..."
9,9,{900: 'No-relief-with-Motrin-OR-no-relief-with...


In [196]:
notes_features2 = patient_notes.merge(right=df, left_on="case_num", right_on="case_num")
notes_features2

,pn_num,case_num,pn_history,features
0,0,0,"17-year-old male, has come to the student heal...",{0: 'Family-history-of-MI-OR-Family-history-of...
1,1,0,17 yo male with recurrent palpitations for the...,{0: 'Family-history-of-MI-OR-Family-history-of...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...,{0: 'Family-history-of-MI-OR-Family-history-of...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...,{0: 'Family-history-of-MI-OR-Family-history-of...
4,4,0,17yo male with no pmh here for evaluation of p...,{0: 'Family-history-of-MI-OR-Family-history-of...
...,...,...,...,...
42141,95330,9,Ms. Madden is a 20 yo female presenting w/ the...,{900: 'No-relief-with-Motrin-OR-no-relief-with...
42142,95331,9,A 20 YO F CAME COMPLAIN A DULL 8/10 HEADACHE T...,{900: 'No-relief-with-Motrin-OR-no-relief-with...
42143,95332,9,Ms. Madden is a 20yo female who presents with ...,{900: 'No-relief-with-Motrin-OR-no-relief-with...
42144,95333,9,Stephanie madden is a 20 year old woman compla...,{900: 'No-relief-with-Motrin-OR-no-relief-with...


### redo cleaning and finding entities

In [197]:
notes_features2["clean_pn_history"] = notes_features2["pn_history"].apply(prepare_jag.basic_clean3).apply(prepare_jag.remove_stopwords, exclude_words=["no"])
notes_features2.head()

,pn_num,case_num,pn_history,features,clean_pn_history
0,0,0,"17-year-old male, has come to the student heal...",{0: 'Family-history-of-MI-OR-Family-history-of...,17-year-old male come student health clinic co...
1,1,0,17 yo male with recurrent palpitations for the...,{0: 'Family-history-of-MI-OR-Family-history-of...,17 yo male recurrent palpitations past 3 mo la...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...,{0: 'Family-history-of-MI-OR-Family-history-of...,dillon cleveland 17 male patient no significan...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...,{0: 'Family-history-of-MI-OR-Family-history-of...,17 yo c/o palpitation started 3 mos ago nothin...
4,4,0,17yo male with no pmh here for evaluation of p...,{0: 'Family-history-of-MI-OR-Family-history-of...,17yo male no pmh evaluation palpitations state...


In [198]:
notes_features2["note_entities"] = notes_features2["clean_pn_history"].apply(get_doc_ents)
notes_features2.head()

,pn_num,case_num,pn_history,features,clean_pn_history,note_entities
0,0,0,"17-year-old male, has come to the student heal...",{0: 'Family-history-of-MI-OR-Family-history-of...,17-year-old male come student health clinic co...,"{'heart pounding': [56, 70], 'dispnea': [257, ..."
1,1,0,17 yo male with recurrent palpitations for the...,{0: 'Family-history-of-MI-OR-Family-history-of...,17 yo male recurrent palpitations past 3 mo la...,"{'palpitations': [21, 33], 'chest catching bre..."
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...,{0: 'Family-history-of-MI-OR-Family-history-of...,dillon cleveland 17 male patient no significan...,"{'complaints heart pounding': [61, 86], 'pain'..."
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...,{0: 'Family-history-of-MI-OR-Family-history-of...,17 yo c/o palpitation started 3 mos ago nothin...,"{'palpitation': [10, 21], 'nausea vomiting hea..."
4,4,0,17yo male with no pmh here for evaluation of p...,{0: 'Family-history-of-MI-OR-Family-history-of...,17yo male no pmh evaluation palpitations state...,"{'palpitations': [28, 40], '1-2 times month pe..."
